# Usando algoritmos genéticos para invertir el juego de la vida

## Sobre el problema

Primer problema: **No es posible invertir el autómata celular del juego de la vida**

- Esto quiere decir que no existen reglas que permitan invertir la flecha temporal
- Hay algunos patrones que no tienen patrón previo, por ejemplo los [jardines del eden](https://www.conwaylife.com/wiki/Garden_of_Eden)
- Hay otros que tienen potencialmente miles de patrones previos, por ejemplo el universo vacío

Esto no quiere decir que no podamos buscar *uno* de los posibles patrones que dan lugar a un detrminado patrón

- O al menos a un patrón suficientemente parecido

Sí existen autómatas celulares _invertibles_ (ver [https://en.wikipedia.org/wiki/Reversible_cellular_automaton](https://en.wikipedia.org/wiki/Reversible_cellular_automaton))

- Por ejemplo el autómata celular [Critters](https://en.wikipedia.org/wiki/Critters_(cellular_automaton)), que se parece bastante al juego de la vida

¿Qué vamos a hacer nosotros?

- Vamos a buscar patrones que tras evolucionar $n$ pasos den un patrón muy parecido a un patrón objetivo
- Vamos a programar un algoritmo geńetico para llegar a dicho patrón.

## Programación del algoritmo genético

Comenzamos importando las librerías que vamos a utilizar a lo largo de la presentación

In [1]:
from itertools import product
from random import choices, random, randrange
from time import sleep

from IPython.display import clear_output

from life import Universe, board, blinker, toad, beehive, shift

Y como a representar el universo como un tablero con topología toroidal de $6 \times 6$ pues lo definimos como constante

In [2]:
UNIVERSE_SIZE = 6, 6

Toroidal: Lo que se sale por un extemo aparece por el otro)

También vamos a determinar cuál es el universo objetivo del que queremos saber un posible estado pasado

In [3]:
TARGET=Universe(shift(toad, 1, 4) | shift(blinker, 2, 1), board_size=UNIVERSE_SIZE)
print(TARGET)

⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬜⬛⬛⬛⬜⬜


### Inicialización

Estableceremos primero la probabilidad de que una célula esté viva

In [4]:
ALIVE_CELL_AT_INIT_PROB = 0.5

Ahora ya podemos crear una inicialización

In [5]:
def gol_initialize(pop_size, gen_size):
    width, height = gen_size
    
    return [
        {(x, y) for x, y in product(range(width), range(height)) if random() < ALIVE_CELL_AT_INIT_PROB}
        for _ in range(pop_size)
    ]

Veamos cómo queda:

In [6]:
population = gol_initialize(pop_size=5, gen_size=UNIVERSE_SIZE)
for i, genotype in enumerate(population):
    print(f'Universe {i+1}:\n{Universe(population[i], board_size=UNIVERSE_SIZE)}')

Universe 1:
⬜⬛⬜⬜⬛⬛
⬜⬛⬛⬛⬛⬜
⬜⬛⬛⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬛⬛⬜⬛⬜⬜
⬛⬜⬛⬛⬛⬜
Universe 2:
⬛⬛⬛⬛⬜⬜
⬛⬛⬜⬜⬛⬜
⬜⬛⬜⬜⬛⬜
⬜⬜⬛⬛⬛⬛
⬛⬜⬜⬜⬛⬜
⬜⬜⬛⬛⬜⬜
Universe 3:
⬜⬛⬜⬜⬜⬛
⬛⬜⬜⬛⬛⬜
⬜⬜⬜⬛⬛⬛
⬜⬛⬛⬜⬛⬜
⬜⬛⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜
Universe 4:
⬛⬜⬜⬛⬜⬜
⬜⬛⬛⬛⬜⬛
⬜⬜⬜⬜⬜⬛
⬛⬛⬛⬜⬛⬜
⬜⬛⬛⬛⬜⬜
⬜⬜⬛⬛⬛⬜
Universe 5:
⬜⬛⬜⬜⬛⬜
⬜⬛⬛⬜⬛⬛
⬜⬜⬛⬛⬛⬛
⬛⬜⬛⬛⬜⬜
⬜⬛⬛⬛⬜⬜
⬜⬜⬜⬜⬜⬜


### Fitness

Nuestro objetivo es determinar cuál es el tablero que, tras $n$ pasos nos da un tablero objetivo

In [7]:
STEPS_IN_FUTURE = 20

Tenemos que determinar una función que nos diga cómo de cerca estamos de ese objetivo

- Diremos que está más cerca cuantas más células coinciden con el estado esperado

In [8]:
def gol_fitness(genotype):
    # Sacamos el futuro de nuestro universo
    universe = Universe(genotype, board_size=UNIVERSE_SIZE)
    for _ in range(STEPS_IN_FUTURE):
        universe.step()
    # El fitness será todas las células que coinciden, es decir, el total menos el cardinal de la diferencia simétrica
    simetric_diff = universe.cells ^ TARGET.cells
    return UNIVERSE_SIZE[0] * UNIVERSE_SIZE[1] - len(simetric_diff)

A ver qué tal la población que hemos generado:

In [9]:
for i, genotype in enumerate(population):
    print(f'Universe {i+1} (fitness {gol_fitness(genotype)}):\n{Universe(population[i], board_size=UNIVERSE_SIZE)}')

Universe 1 (fitness 27):
⬜⬛⬜⬜⬛⬛
⬜⬛⬛⬛⬛⬜
⬜⬛⬛⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬛⬛⬜⬛⬜⬜
⬛⬜⬛⬛⬛⬜
Universe 2 (fitness 23):
⬛⬛⬛⬛⬜⬜
⬛⬛⬜⬜⬛⬜
⬜⬛⬜⬜⬛⬜
⬜⬜⬛⬛⬛⬛
⬛⬜⬜⬜⬛⬜
⬜⬜⬛⬛⬜⬜
Universe 3 (fitness 27):
⬜⬛⬜⬜⬜⬛
⬛⬜⬜⬛⬛⬜
⬜⬜⬜⬛⬛⬛
⬜⬛⬛⬜⬛⬜
⬜⬛⬜⬜⬜⬛
⬜⬜⬜⬜⬛⬜
Universe 4 (fitness 19):
⬛⬜⬜⬛⬜⬜
⬜⬛⬛⬛⬜⬛
⬜⬜⬜⬜⬜⬛
⬛⬛⬛⬜⬛⬜
⬜⬛⬛⬛⬜⬜
⬜⬜⬛⬛⬛⬜
Universe 5 (fitness 21):
⬜⬛⬜⬜⬛⬜
⬜⬛⬛⬜⬛⬛
⬜⬜⬛⬛⬛⬛
⬛⬜⬛⬛⬜⬜
⬜⬛⬛⬛⬜⬜
⬜⬜⬜⬜⬜⬜


### Operadores

#### Condición de parada, selección y reemplazo

Usaremos los que ya vimos en la anterior presentación, número de generaciones, torneo y generacional respectivamente

In [10]:
def num_generations_stop(target_generation):
    def f(population, generation):
        return generation >= target_generation
    return f


def tournament_selection(t):
    def f(pop, n, fitness):
        return [max(choices(pop, k=t), key=fitness) for _ in range(n)]
    return f


def generational_replacement(old_population, new_population):
    return new_population

#### Recombinación

El operador de recombinación que usaremos aquí será similar al de la anterior presentación

- Antes, usamos un punto de pivote para determinar las mitades
- Ahora usaremos un $(x, y)$ de pivote que partirá el tablero en dos mitades

In [11]:
def gol_recombination(parent1, parent2):
    pivot_x, pivot_y = randrange(UNIVERSE_SIZE[0]), randrange(UNIVERSE_SIZE[1])
    child1, child2 = parent1.copy(), parent2.copy()
    
    width, height = UNIVERSE_SIZE
    for cell in product(range(pivot_x, width), range(pivot_y, height)):
        if cell in parent1:
            child2.add(cell)
        else:
            child2.discard(cell)
        if cell in parent2:
            child1.add(cell)
        else:
            child1.discard(cell)
    
    return child1, child2

Veamos cómo se comporta el cruce:

In [12]:
children = gol_recombination(population[0], population[1])
for i in range(2):
    print(f'Parent {i+1}:\n{Universe(population[i], UNIVERSE_SIZE)}')
for i in range(2):
    print(f'Child {i+1}:\n{Universe(children[i], UNIVERSE_SIZE)}')

Parent 1:
⬜⬛⬜⬜⬛⬛
⬜⬛⬛⬛⬛⬜
⬜⬛⬛⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬛⬛⬜⬛⬜⬜
⬛⬜⬛⬛⬛⬜
Parent 2:
⬛⬛⬛⬛⬜⬜
⬛⬛⬜⬜⬛⬜
⬜⬛⬜⬜⬛⬜
⬜⬜⬛⬛⬛⬛
⬛⬜⬜⬜⬛⬜
⬜⬜⬛⬛⬜⬜
Child 1:
⬜⬛⬛⬛⬜⬜
⬜⬛⬜⬜⬛⬜
⬜⬛⬜⬜⬛⬜
⬜⬜⬛⬛⬛⬛
⬛⬛⬜⬜⬛⬜
⬛⬜⬛⬛⬜⬜
Child 2:
⬛⬛⬜⬜⬛⬛
⬛⬛⬛⬛⬛⬜
⬜⬛⬛⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬛⬜⬜⬛⬜⬜
⬜⬜⬛⬛⬛⬜


#### Mutación

En este caso, también haremos una mutación similar a la que hicimos en la anterior presentación

Esto es, recorreremos todas las células y con con cierta probabilidad cambiamos su estado

In [13]:
def gol_mutation(genotype, p_mutation):
    mutated = genotype.copy()

    width, height = UNIVERSE_SIZE
    for x, y in product(range(width), range(height)):
        if random() < p_mutation:
            if (x, y) in genotype:
                mutated.remove((x, y))
            else:
                mutated.add((x, y))
    
    return mutated

A ver cómo queda un tablero mutado

In [14]:
mutated = gol_mutation(TARGET.cells, p_mutation=1)
print(f'Original:\n{Universe(mutated, board_size=UNIVERSE_SIZE)}')

Original:
⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬜⬜⬛
⬛⬛⬛⬛⬛⬛
⬛⬛⬛⬛⬛⬛
⬛⬛⬜⬜⬜⬛
⬛⬜⬜⬜⬛⬛


### Algoritmo genético

Ya tenemos todas las piezas y podemos ejecutar nuestro algoritmo genético; primero definamos una serie de variables de configuración

In [15]:
NUM_GENERATIONS = 100  # Número de iteraciones del algoritmo
POPULATION_SIZE = 100  # Tamaño de la población
TOURNAMENT_SIZE = 2   # Tamaño de la muestra en el algoritmo de torneo

p_recombination = 0.9  # Probabilidad de cruce
p_mutation = 5. / (UNIVERSE_SIZE[0] * UNIVERSE_SIZE[1])  # Probabilidad de mutación

stop = num_generations_stop(NUM_GENERATIONS)
select = tournament_selection(t=TOURNAMENT_SIZE)
fitness = gol_fitness
initialize = gol_initialize
recombine = gol_recombination
mutate = gol_mutation
replace = generational_replacement

Y segundo, ejecutemos el algoritmo

In [16]:
# Inicializamos la población
population = initialize(POPULATION_SIZE, UNIVERSE_SIZE)

# Vamos iterando sobre generaciones hasta cumplir la condición de parada
generation = 0
while not stop(population, generation):
    print('.', end='')
    # Creamos la nueva descendencia (en principio vacía)
    offspring = []
    # La rellenamos realizando operaciones de selección + cruce + mutación
    while len(offspring) < len(population):
        # Selección
        genotype1, genotype2 = select(population, n=2, fitness=fitness)
        # Recombinación
        if random() < p_recombination:
            genotype1, genotype2 = recombine(genotype1, genotype2)
        # Mutación
        genotype1 = mutate(genotype1, p_mutation)
        genotype2 = mutate(genotype2, p_mutation)
        # Los añadimos a la descendencia
        offspring.append(genotype1)
        # Para no meter uno de más si el tamaño de la población es impar
        if len(offspring) < len(population):
            offspring.append(genotype2)
    
    # Reemplazamos la anterior población por la nueva
    population = replace(population, offspring)
    # Incrementamos la generación
    generation += 1

....................................................................................................

Veamos que solución ha encontrado:

In [17]:
best_genome = max(population, key=fitness)

universe = Universe(best_genome, UNIVERSE_SIZE)
print(f'Source universe (fitness = {fitness(best_genome)}):\n{universe}')
for _ in range(STEPS_IN_FUTURE):
    universe.step()
print(f'Evolved universe after {STEPS_IN_FUTURE} steps:\n{universe}')
print(f'Target universe\n{TARGET}')

Source universe (fitness = 29):
⬜⬜⬛⬛⬜⬛
⬛⬜⬛⬛⬜⬛
⬜⬛⬛⬛⬛⬜
⬜⬜⬜⬛⬜⬜
⬜⬜⬛⬜⬛⬛
⬜⬛⬛⬛⬜⬜
Evolved universe after 20 steps:
⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜
⬜⬛⬛⬜⬜⬜
⬜⬛⬛⬜⬜⬜
Target universe
⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬜⬜⬜⬜⬜⬜
⬜⬜⬜⬜⬜⬜
⬜⬜⬛⬛⬛⬜
⬜⬛⬛⬛⬜⬜


## Finalizando

Como podéis imaginar, no es una tarea sencilla

Tanto es así que han existido no una, sino dos competiciones en Kaggle sobre este tema

- Competición de 2013: [https://www.kaggle.com/c/conway-s-reverse-game-of-life](https://www.kaggle.com/c/conway-s-reverse-game-of-life)
- Competición de 2020: [https://www.kaggle.com/c/conways-reverse-game-of-life-2020](https://www.kaggle.com/c/conways-reverse-game-of-life-2020)

Las técnicas que se han usado son de lo más variopinto:

- Redes de convolución
- Algoritmos genéticos
- Recocido simulado

Esto ha sido únicamente un pequeño ejemplo de a lo que se puede llegar; ¿os atrevéis?

# ¡Muchas gracias!